<a href="https://colab.research.google.com/github/msdsn/GaitLab/blob/main/ConvLstm_(LR%3D0_05)_Fusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

In [ ]:
!gdown 1d-DGCtaex6YZNg2ad2aPzRWW0xn4bVj3
!unzip "/content/GaitDatasetB-silh.zip" -d "/content/GaitDatasetB"

Downloading...
From (original): https://drive.google.com/uc?id=1d-DGCtaex6YZNg2ad2aPzRWW0xn4bVj3
From (redirected): https://drive.google.com/uc?id=1d-DGCtaex6YZNg2ad2aPzRWW0xn4bVj3&confirm=t&uuid=e13f2461-6b52-4d5e-8abf-7eca141da97b
To: /content/GaitDatasetB-silh.zip
100% 659M/659M [00:03<00:00, 183MB/s]
Archive:  /content/GaitDatasetB-silh.zip
   creating: /content/GaitDatasetB/GaitDatasetB-silh/
 extracting: /content/GaitDatasetB/GaitDatasetB-silh/001.tar.gz  
 extracting: /content/GaitDatasetB/GaitDatasetB-silh/002.tar.gz  
 extracting: /content/GaitDatasetB/GaitDatasetB-silh/003.tar.gz  
 extracting: /content/GaitDatasetB/GaitDatasetB-silh/004.tar.gz  
 extracting: /content/GaitDatasetB/GaitDatasetB-silh/005.tar.gz  
 extracting: /content/GaitDatasetB/GaitDatasetB-silh/006.tar.gz  
 extracting: /content/GaitDatasetB/GaitDatasetB-silh/007.tar.gz  
 extracting: /content/GaitDatasetB/GaitDatasetB-silh/008.tar.gz  
 extracting: /content/GaitDatasetB/GaitDatasetB-silh/009.tar.gz  
 extr

In [ ]:
import tarfile
import os

def unzip_tar_gz(folder_path):
  for filename in os.listdir(folder_path):
    if filename.endswith(".tar.gz"):
      filepath = os.path.join(folder_path, filename)
      try:
        with tarfile.open(filepath, "r:gz") as tar:
          tar.extractall(folder_path)
          #print(f"Extracted: {filename}")
        # remove tar.gz
        os.remove(filepath)
      except Exception as e:
        print(f"Error extracting {filename}: {e}")

unzip_tar_gz("/content/GaitDatasetB/GaitDatasetB-silh")

In [ ]:
!git clone https://github.com/ShiqiYu/OpenGait.git

Cloning into 'OpenGait'...
remote: Enumerating objects: 1848, done.
remote: Counting objects: 100% (485/485), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 1848 (delta 385), reused 362 (delta 344), pack-reused 1363 (from 1)
Receiving objects: 100% (1848/1848), 20.23 MiB | 40.29 MiB/s, done.
Resolving deltas: 100% (1163/1163), done.


In [ ]:
!python "/content/OpenGait/datasets/pretreatment.py" --input_path "/content/GaitDatasetB/GaitDatasetB-silh" --output_path CASIA-B-pkl

Pretreating: 100% 13593/13593 [08:09<00:00, 27.76folder/s]


In [ ]:
pip install tqdm pyyaml tensorboard opencv-python kornia einops torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.4/899.4 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.2 MB/s eta 0:00:00


In [ ]:
import os
import pickle
import os.path as osp
import json
import copy
from collections import OrderedDict


import math
import random
import numpy as np
from scipy.spatial.distance import cdist

import torch
import torch.nn as nn
from torch import Tensor
import torch.utils.data as tordata
import torch.nn.functional as F

from torchvision.models.resnet import ResNet, BasicBlock

In [ ]:
class DataSet(tordata.Dataset):
    def __init__(self, data_set):
        """
            seqs_info: the list with each element indicating
                            a certain gait sequence presented as [label, type, view, paths];
        """
        self.data_set = data_set
        self.__dataset_parser()
        self.label_list = [seq_info[0] for seq_info in self.seqs_info]
        self.types_list = [seq_info[1] for seq_info in self.seqs_info]
        self.views_list = [seq_info[2] for seq_info in self.seqs_info]

        self.label_set = sorted(list(set(self.label_list)))
        self.types_set = sorted(list(set(self.types_list)))
        self.views_set = sorted(list(set(self.views_list)))
        self.seqs_data = [None] * len(self)
        self.indices_dict = {label: [] for label in self.label_set}
        for i, seq_info in enumerate(self.seqs_info):
            self.indices_dict[seq_info[0]].append(i)

    def __len__(self):
        return len(self.seqs_info)

    def __loader__(self, pth):
        if pth.endswith('.pkl'):
            with open(pth, 'rb') as f:
                data = pickle.load(f)
            f.close()
        return data
    def __getitem__(self, idx):
        if self.seqs_data[idx] is None:
            data = self.__loader__(self.seqs_info[idx][-1])
            self.seqs_data[idx] = data
        else:
            data = self.seqs_data[idx]
        seq_info = self.seqs_info[idx]
        return data, seq_info

    def __load_all_data(self):
        for idx in range(len(self)):
            self.__getitem__(idx)

    def __dataset_parser(self):
        dataset_root = '/content/CASIA-B-pkl'


        def get_seqs_info_list(label_set):
            seqs_info_list = []
            for lab in label_set:
                for typ in sorted(os.listdir(osp.join(dataset_root, lab))):
                    for vie in sorted(os.listdir(osp.join(dataset_root, lab, typ))):
                        seq_info = [lab, typ, vie]
                        seq_path = osp.join(dataset_root, *seq_info)
                        dir = os.listdir(seq_path)
                        flag = False
                        for file in dir:
                            if file.endswith(".pkl"):
                                pth = osp.join(seq_path, file)
                                seqs_info_list.append([*seq_info, pth])
                                flag = True
                                break;
                        if not flag:
                            print(
                                'Find no .pkl file in %s-%s-%s.' % (lab, typ, vie))
            return seqs_info_list

        self.seqs_info = get_seqs_info_list(self.data_set) #if training else get_seqs_info_list(test_set)

In [ ]:
class CollateFn(object):
    def __init__(self, label_set):
        self.label_set = label_set
        self.frames_num = 30

    def __call__(self, batch):
        # list of (data, seq_info), data if array of pkl files in a view, seq_info is => (lab, type, view, paths-(pkl files))
        batch_size = len(batch)
        seqs_batch, labs_batch, typs_batch, vies_batch = [], [], [], []
        for bt in batch:
            seq_len = len(bt[0])
            replace= seq_len < self.frames_num
            indices = list(range(seq_len))
            indices = np.random.choice(indices, size=self.frames_num, replace=replace)
            # numpy.ndarray with numpy.array() before converting to a tensor
            seq_numpy = np.array([bt[0][i] for i in indices])
            seq_tensor = torch.tensor(seq_numpy)
            seqs_batch.append(seq_tensor)
            lab_numpy = np.array([self.label_set.index(bt[1][0])])
            lab_tensor = torch.tensor(lab_numpy)
            labs_batch.append(lab_tensor)
            typs_batch.append(bt[1][1])
            vies_batch.append(bt[1][2])
        seqs_batch = torch.stack(seqs_batch).float()
        seqs_batch = seqs_batch.unsqueeze(2)
        seqs_batch = seqs_batch / 255.0

        labs_batch = torch.stack(labs_batch).squeeze(1).long()

        batch = [seqs_batch, labs_batch, typs_batch, vies_batch]
        return batch

In [ ]:
import math
import random
import torch.utils.data as tordata
class TripletSampler(tordata.sampler.Sampler):
    def __init__(self, dataset, batch_size, batch_shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        if len(self.batch_size) != 2:
            raise ValueError(
                "batch_size should be (P x K) not {}".format(batch_size))
        self.batch_shuffle = batch_shuffle
        self.world_size = 1
        self.rank = 0

    def __iter__(self):
        while True:
            sample_indices = []
            pid_list = random_sample_list(
                self.dataset.label_set, self.batch_size[0])

            for pid in pid_list:
                indices = self.dataset.indices_dict[pid]
                indices = random_sample_list(
                    indices, k=self.batch_size[1])
                sample_indices += indices

            if self.batch_shuffle:
                sample_indices = random_sample_list(
                    sample_indices, len(sample_indices))

            total_batch_size = self.batch_size[0] * self.batch_size[1]
            sample_indices += sample_indices[:(total_batch_size - len(sample_indices))]
            yield sample_indices

    def __len__(self):
        return len(self.dataset)

def random_sample_list(obj_list, k):
    if len(obj_list) < k:
        idx = random.choices(range(len(obj_list)), k=k)
    else:
        idx = random.sample(range(len(obj_list)), k=k)
    return [obj_list[i] for i in idx]

In [ ]:
with open('/content/OpenGait/datasets/CASIA-B/CASIA-B.json', "rb") as f:
    partition = json.load(f)
train_set = partition["TRAIN_SET"]
test_set = partition["TEST_SET"]
label_list = os.listdir('/content/CASIA-B-pkl')
train_set = [label for label in train_set if label in label_list]
test_set = [label for label in test_set if label in label_list]

In [ ]:
dataset = DataSet(data_set=train_set)
sampler = TripletSampler(dataset, batch_size=[8, 16])
collate_fn = CollateFn(dataset.label_set)
train_loader = tordata.DataLoader(dataset, batch_sampler=sampler, collate_fn=collate_fn, num_workers=2)

In [ ]:
for batch in train_loader:
  seqs, labs, types, views = batch
  print(seqs.shape) # (n, s, c, h, w)
  print(labs.shape)
  break

torch.Size([128, 30, 1, 64, 64])
torch.Size([128])


# Create Your Own Model Here:

In [ ]:
class HorizontalPoolingPyramid():
    """
        Horizontal Pyramid Matching for Person Re-identification
        Arxiv: https://arxiv.org/abs/1804.05275
        Github: https://github.com/SHI-Labs/Horizontal-Pyramid-Matching
    """

    def __init__(self):
        self.bin_num = [16]

    def __call__(self, x):
        """
            x  : [n, c, h, w]
            ret: [n, c, p]
        """
        n, c = x.size()[:2]
        features = []
        for b in self.bin_num:
            z = x.view(n, c, b, -1)
            z = z.mean(-1) + z.max(-1)[0]
            features.append(z)
        return torch.cat(features, -1)

### Resnetin çıktısı: `n, s, c, h, w`
### Biz bu verktörü `(n * h * w), s, c` olarak grupladığımız zaman şunu sağlamış oluyoruz:

Her sampledaki her spatial location için frame boyunca kanal bilgisi.

`Yani yürüyüşlerdeki her piksel için zaman x kanal(512)` bilgisi.

### Bunu LSTM ne yapabilir.

LSTM sonunda biz her Spatial Location(h,w) için, Sequencetaki Temporal Dependenci'leri yakalamış oluruz.

In [ ]:
class CustomModel(ResNet):
    def __init__(self):
      super(CustomModel, self).__init__(BasicBlock, [1, 1, 1, 1])
      layers = [1, 1, 1, 1]
      self.inplanes = 64
      self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
      self.layer1 = self._make_layer(BasicBlock, 64, layers[0])
      self.layer2 = self._make_layer(BasicBlock, 128, layers[1], dilate=False)
      self.layer3 = self._make_layer(BasicBlock, 256, layers[2], dilate=False)
      self.layer4 = self._make_layer(BasicBlock, 512, layers[3], dilate=False)
      self.lstm = nn.LSTM(input_size=512, hidden_size=256, num_layers=1, batch_first=True)
      self.fusion_conv = nn.Conv2d(512+256, 256, kernel_size=1, stride=1, padding=0, bias=False)
      self.hpp = HorizontalPoolingPyramid()

    def forward(self, x: Tensor) -> Tensor:
        # x = torch.Size([128*30, 1, 64, 64]) (n*s, c, h, w)
        x = self.conv1(x) # torch.Size([3840, 64, 32, 32]) (n*s, c=64, h, w)
        x = self.bn1(x) # torch.Size([3840, 64, 32, 32])
        x = self.relu(x) # torch.Size([3840, 64, 32, 32])
        x = self.maxpool(x) # torch.Size([3840, 64, 16, 16])

        x = self.layer1(x) # torch.Size([3840, 64, 16, 16])
        x = self.layer2(x) # torch.Size([3840, 128, 16, 16])
        x = self.layer3(x) # torch.Size([3840, 256, 16, 16])
        x = self.layer4(x) # torch.Size([3840, 512, 16, 16]) (n*s, c=512, h, w)

        n_s, c, h, w = x.size()

        # --- LSTM Yolu ---
        x1 = x.view(128, 30, c, h, w) # (n, s, c, h, w) = torch.Size([128, 30, 512, 16, 16])
        # Permute to [n, h, w, s, c]
        x1 = x1.permute(0, 3, 4, 1, 2)  # [n, h, w, s, c]
        n, h, w, s, c = x1.shape
        # Flatten batch and spatial dimensions
        x1 = x1.contiguous().view(n * h * w, s, c)  # [n*h*w, s, c]
        # Pass through LSTM
        x1, _ = self.lstm(x1)  # x: [n*h*w, s, hidden_size]
        # Get embedding from the last time step
        x1 = x1[:, -1, :]  # [n*h*w, hidden_size]
        # Reshape back to [n, h, w, hidden_size]
        x1 = x1.view(n, h, w, -1)
        # Permute to [n, hidden_size, h, w]
        x1 = x1.permute(0, 3, 1, 2)  # [n, hidden_size=256, h, w]

        # --- Torch Max Yolu ---
        x2 = x.reshape(128, 30, 512, 16, 16).transpose(1, 2).contiguous()
        x2 = torch.max(x2, dim=2)[0]  # (n, c, h, w)

        # --- Füzyon ---
        x_concat = torch.cat((x1, x2), dim=1)  # (n, c + hidden_size, h, w)
        x_fused = self.fusion_conv(x_concat)

        x = self.hpp(x_fused) # # [n, c=256, p=16]

        return x



# Loss

In [ ]:
# Iterative Triplet Loss Implementation
class TripletLossIterative(nn.Module):
    def __init__(self, margin=0.2):
        super(TripletLossIterative, self).__init__()
        self.margin = margin

    def forward(self, embeddings, labels):
        n, c, p = embeddings.size()
        embeddings = embeddings.permute(2, 0, 1).contiguous().float()  # [p, n, c]
        device = embeddings.device
        total_loss = 0.0
        total_triplets = 0

        for part in range(p):  # Loop over each part
            emb_part = embeddings[part]  # [n, c]
            dist_mat = torch.cdist(emb_part, emb_part, p=2)  # [n, n]

            for i in range(n):  # Loop over each anchor
                anchor_label = labels[i]
                anchor_dist = dist_mat[i]  # Distances from anchor to all samples

                # Positive and negative indices
                pos_indices = torch.where((labels == anchor_label) & (torch.arange(n).to(device) != i))[0]
                neg_indices = torch.where(labels != anchor_label)[0]

                if len(pos_indices) == 0 or len(neg_indices) == 0:
                    continue  # Skip if no valid positives or negatives

                ap_distances = anchor_dist[pos_indices]  # Distances to positives
                an_distances = anchor_dist[neg_indices]  # Distances to negatives

                # Compute losses for all combinations of positives and negatives
                ap_expanded = ap_distances.unsqueeze(1)  # [num_pos, 1]
                an_expanded = an_distances.unsqueeze(0)  # [1, num_neg]
                loss_values = F.relu(ap_expanded - an_expanded + self.margin)  # [num_pos, num_neg]

                total_loss += loss_values.sum()
                total_triplets += loss_values.numel()

        if total_triplets > 0:
            triplet_loss = total_loss / total_triplets
        else:
            triplet_loss = torch.tensor(0.0, requires_grad=True).to(device)
        return triplet_loss

# Iteration

In [ ]:
test_dataset = DataSet(data_set=test_set)
test_collate_fn = CollateFn(test_dataset.label_set)
test_loader = tordata.DataLoader(test_dataset, collate_fn=test_collate_fn, num_workers=2, drop_last=True, batch_size=128)

In [ ]:
def eval(probe_seq, gallery_seq, features, labels, views, seq_types):

    view_list = np.unique(views)
    acc_list = []

    for probe_view in view_list:
        # Create masks for probe and gallery samples
        probe_mask = np.isin(seq_types, probe_seq) & (views == probe_view)
        gallery_mask = np.isin(seq_types, gallery_seq)

        probe_features = features[probe_mask]  # [num_probe_samples, feature_dim, num_parts]
        probe_labels = labels[probe_mask]

        gallery_features = features[gallery_mask]
        gallery_labels = labels[gallery_mask]
        gallery_views = views[gallery_mask]

        # Exclude gallery samples with the same view as the probe
        non_identical_view_mask = gallery_views != probe_view
        gallery_features = gallery_features[non_identical_view_mask]
        gallery_labels = gallery_labels[non_identical_view_mask]

        if len(probe_features) == 0:
            continue  # Skip if no probe samples for this view

        num_parts = probe_features.shape[2]
        distances_per_part = []

        for part in range(num_parts):
            # Extract features for the current part
            probe_part_features = probe_features[:, :, part]  # [num_probe_samples, feature_dim]
            gallery_part_features = gallery_features[:, :, part]  # [num_gallery_samples, feature_dim]

            # Compute distances for this part
            distances_part = cdist(probe_part_features, gallery_part_features, metric='euclidean')  # [num_probe_samples, num_gallery_samples]
            distances_per_part.append(distances_part)

        # Aggregate distances over parts (e.g., mean)
        distances = np.mean(distances_per_part, axis=0)  # [num_probe_samples, num_gallery_samples]

        # Find nearest neighbors
        min_indices = np.argmin(distances, axis=1)  # [num_probe_samples]
        predicted_labels = gallery_labels[min_indices]  # [num_probe_samples]

        # Compute accuracy
        correct = (predicted_labels == probe_labels)
        acc = np.mean(np.array(correct))
        acc_list.append(acc)

    # Compute mean accuracy over all views
    mean_acc = np.mean(acc_list) if acc_list else 0.0
    return mean_acc

In [ ]:
from einops import rearrange
from torch.cuda.amp import autocast
from torch.cuda.amp import GradScaler
from torch.optim import SGD

model = CustomModel()
if torch.cuda.is_available():
    model = model.cuda()

scaler = GradScaler()
optimizer = SGD(
    model.parameters(),
    lr=0.05,
    momentum=0.9,
    weight_decay=0.0005
)
loss_fn = TripletLossIterative()
iter = 0
losses = np.array([])
for batch in train_loader:
  seqs, labs, _, _ = batch
  if torch.cuda.is_available():
    seqs = seqs.cuda()
    labs = labs.cuda()
  optimizer.zero_grad()
  with autocast():
    seqs = seqs.reshape(-1, 1, 64, 64)
    output = model(seqs)

  loss = loss_fn(output, labs)
  scaler.scale(loss).backward()
  scaler.step(optimizer)
  scaler.update()
  torch.cuda.empty_cache()
  losses = np.append(losses, loss.item())
  iter += 1
  if iter % 10 == 0:
    print(f"Mean loss for {iter} iterations: {losses.mean()}")
  if iter % 20 == 0:
    model.eval()
    features = []
    labels = []
    seq_types = []
    views = []
    with torch.no_grad():
      for batch in test_loader:
        test_seqs, test_labs, batch_types, batch_views = batch
        if torch.cuda.is_available():
          test_seqs = test_seqs.cuda()
          test_labs = test_labs.cuda()
        test_seqs = test_seqs.reshape(-1, 1, 64, 64)
        test_output = model(test_seqs)
        features.append(test_output.cpu())
        labels.append(test_labs.cpu())
        seq_types.append(batch_types)
        views.append(batch_views)
    features = torch.cat(features, dim=0)
    labels = torch.cat(labels, dim=0)
    views = np.array(views).reshape(-1)
    seq_types = np.array(seq_types).reshape(-1)
    gallery_seq = ['nm-01', 'nm-02', 'nm-03', 'nm-04']
    probe_seq = ['bg-01', 'bg-02']
    acc = eval(probe_seq, gallery_seq, features, labels, views, seq_types)
    print(f"Accuracy Bag: {acc}")
    probe_seq = ['nm-05', 'nm-06']
    acc = eval(probe_seq, gallery_seq, features, labels, views, seq_types)
    print(f"Accuracy NM: {acc}")
    probe_seq = ['cl-01','cl-02']
    acc = eval(probe_seq, gallery_seq, features, labels, views, seq_types)
    print(f"Accuracy CL: {acc}")
    model.train()

<ipython-input-19-7fe9c5d0f0b5>:10: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
<ipython-input-19-7fe9c5d0f0b5>:26: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Mean loss for 10 iterations: 0.3281306281685829
Mean loss for 20 iterations: 0.2479355439543724
Accuracy Bag: 0.480011754211781
Accuracy NM: 0.7003710575139147
Accuracy CL: 0.16512059369202223
Mean loss for 30 iterations: 0.2089890661338965
Mean loss for 40 iterations: 0.18511469271034003
Accuracy Bag: 0.6268005881742661
Accuracy NM: 0.8089053803339518
Accuracy CL: 0.23562152133580708
Mean loss for 50 iterations: 0.1683318692445755
Mean loss for 60 iterations: 0.1577143780887127
Accuracy Bag: 0.7271877198239497
Accuracy NM: 0.9044526901669759
Accuracy CL: 0.32653061224489793
Mean loss for 70 iterations: 0.14813393216047968
Mean loss for 80 iterations: 0.13907563630491496
Accuracy Bag: 0.7270844550935593
Accuracy NM: 0.9128014842300556
Accuracy CL: 0.3283858998144713
Mean loss for 90 iterations: 0.13303629437254536
Mean loss for 100 iterations: 0.12762538868933915
Accuracy Bag: 0.7225229360722734
Accuracy NM: 0.9248608534322821
Accuracy CL: 0.34879406307977734
Mean loss for 110 iteratio

KeyboardInterrupt: 

Lr=0.1 için resnet9 modeli fazla büyük bir model olmadığı ve parametre sayısı az olduğu için çok dallanma göstermeyebiliyor ve her model çalıştırıldığında farklı sonuçlar elde edilmiyor. Model varacağı max acc noktasına ulaşabiliyor.

resnet9 + lstm kullanildiginda model bir calistiginda cok iyi sonuc ve baska gun calistirildiginda cok daha dusuk sonuclar verebiliyor.
referans1 resim=> day1 (resnet9 + lstm) for 2000 iter
referans2 resim=> day2 (resnet9 + lstm) for 2000 iter
bu iki resimde goruldugu uzere lr=0.1 oldugunda iki model arasinda cok fark var

ref3 resim => resnet for 2000 iter
ref3 vs ref3 te gorundugu uzere resnet9 + lstm daha iyi sonuc veriyor ama ayni modeli baska zaman calistirdigimizda cok daha dusuk rakamlar elde ediliyor.

fikir resnet9 + lstm modelinde lr=0.01 yapilirsa model kendini dogru sekilde egitebilir mi?

ref4 => (resnet9 + lstm) for 2000 iter with lr=0.01

ref1 vs ref4 de goruldugu uzere lr=0.01 oldugunda dagilmalar cok olmuyor ve daha duz sekilde model egitiliyor.

loss dususu cok daha yavas oldugu goruluyor, bu bize biraz overfit olma durumunu sorgulatiyor.

ResNet9 modeli, parametre sayısının az olması ve modelin çok büyük olmaması nedeniyle, öğrenme oranı (lr) 0.1 olduğunda fazla dallanma göstermiyor ve her çalıştırıldığında benzer sonuçlar elde ediliyor. Model, ulaşabileceği maksimum doğruluk noktasına hızlıca erişebiliyor.

Ancak, ResNet9 modeline LSTM eklendiğinde, modelin performansı tutarsızlık gösterebiliyor. Bir çalıştırmada çok iyi sonuçlar verirken, başka bir gün aynı model çok daha düşük sonuçlar verebiliyor. Bu durumu aşağıdaki referanslarla inceledik:

*	Referans 1: Day 1 - ResNet9 + LSTM, 2000 iterasyon, lr=0.1
*	Referans 2: Day 2 - ResNet9 + LSTM, 2000 iterasyon, lr=0.1

Bu iki referansta görüldüğü üzere, aynı model ve öğrenme oranı kullanıldığında bile sonuçlar arasında büyük farklılıklar oluşuyor.

Ayrıca, sadece ResNet9 modelini incelediğimizde:
*	Referans 3: ResNet9, 2000 iterasyon, lr=0.1

Referans 3 ile ResNet9 + LSTM modellerini karşılaştırdığımızda, LSTM eklenmiş modelin genellikle daha iyi sonuçlar verdiğini gözlemliyoruz. Ancak, bu modelin farklı zamanlarda tutarsız performans sergilemesi, öğrenme oranının yüksek olmasından kaynaklanıyor olabilir.

Bu nedenle, şu soruyu sormaya başladık: “ResNet9 + LSTM modelinde öğrenme oranı lr=0.01 olarak ayarlanırsa, model kendini daha tutarlı ve doğru bir şekilde eğitebilir mi?”

Bu hipotezi test etmek için:
* Referans 4: ResNet9 + LSTM, 2000 iterasyon, lr=0.01

Referans 1 ile Referans 4'ü karşılaştırdığımızda, lr=0.01 olduğunda modelin eğitim sürecinin daha stabil olduğunu ve aşırı dalgalanmaların azaldığını görüyoruz. Eğitim eğrileri daha düz bir şekilde ilerliyor, ancak loss değeri düşüşü daha yavaş gerçekleşiyor. Bu durum, modelin bir yerlerde uzun süre tıkalı kalma riskini düşündürüyor ve daha uzun bir eğitim süresine ihtiyaç duyabileceğini gösteriyor. 2000 iterasyon için bu karşılaştırmayı yapmak adil olmadığı için lr=0.01 olduğu zaman 3000 iterasyon yaparak bir karşılaştırmaya gidelim:
* Referans 5: (ResNet9 + LSTM, 3000 iterasyon, lr=0.01) vs (ResNet9 + LSTM, 2000 iterasyon, lr=0.1)

Sonuç olarak, öğrenme oranının düşürülmesi, ResNet9 + LSTM modelinin daha tutarlı bir performans sergilemesine yardımcı olabilir. Ancak, bu ayarın loss düşüş hızını etkilediği ve potansiyel olarak overfitting riskini artırdığı da göz önünde bulundurulmalıdır. 2000 iterasyon için bu şekilde bir durum söz konusu değil.

Yukarıdaki analiz, çeşitli log ve grafiklerle desteklenmiştir ve modeller arasındaki farkları daha net bir şekilde ortaya koymayı amaçlamaktadır.